# Домашнее задание № 3

Создание / стилизация видео

Ваша задача — создать нейросетевое видео с использованием одной из двух технологий, показанных в лекции:
— Покадровая обработка существующего видео с применением ControlNet (предпочтительно) или Image-to-Image
— Генерация видео с помощью латентной интерполяции из двух и более (желательно) текстовых запросов.

Продолжительность видео — не менее 5 секунд (15 кадров в секунду).


Критерии одобрения:
— Оригинальность идеи
— Качество исполнения
— Сложность исполнения (использование ControlNet/нескольких промптов)


## Скачивание данных

In [1]:
# Пакет для установки датасетов из Huggingface
!pip install -q datasets "huggingface_hub[cli]" torchcodec

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("UniDataPro/deepfake-videos-dataset", split='train')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
type(ds[0]['video'])

torchcodec.decoders._video_decoder.VideoDecoder

## Проебразования видео

In [4]:
!pip install -q transformers==4.49 diffusers==0.32.2 accelerate opencv-python pillow mediapy
!pip install -q controlnet_aux xformers safetensors tqdm
!apt-get -qq install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.0 MB/s eta 0:00:00


In [5]:
import torch
import cv2
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import mediapy as media
from IPython.display import display, Video
import gc

In [6]:
def extract_frames_from_decoder(video_decoder, max_frames=None, target_fps=15, resize_width=512):
    """
    Извлекает кадры из torchcodec VideoDecoder.

    Args:
        video_decoder: VideoDecoder объект
        max_frames: максимальное количество кадров (None = все)
        target_fps: целевой FPS для сэмплирования
        resize_width: ширина для изменения размера кадров
    """
    try:
        # Получаем метаданные видео
        # metadata это VideoStreamMetadata объект, не словарь
        metadata = video_decoder.metadata if hasattr(video_decoder, 'metadata') else None

        # Получаем общее количество кадров и FPS
        if metadata is not None:
            # Обращаемся к атрибутам, а не как к словарю
            total_frames = metadata.num_frames if hasattr(metadata, 'num_frames') else 300
            original_fps = metadata.average_fps if hasattr(metadata, 'average_fps') else 30.0
        else:
            total_frames = 300
            original_fps = 30.0

        print(f"Исходное видео: {original_fps:.2f} FPS, {total_frames} кадров")

        # Вычисляем шаг для достижения целевого FPS
        frame_step = max(1, int(original_fps / target_fps))

        # Ограничиваем количество кадров
        if max_frames is None:
            num_frames_to_extract = total_frames // frame_step
        else:
            num_frames_to_extract = min(max_frames, total_frames // frame_step)

        frames = []

        print(f"Будет извлечено {num_frames_to_extract} кадров с шагом {frame_step}")

        for i in tqdm(range(num_frames_to_extract), desc="Извлечение кадров"):
            frame_idx = i * frame_step

            # Проверяем, не выходим ли за границы
            if frame_idx >= total_frames:
                break

            try:
                # Получаем кадр из VideoDecoder
                # Обычно torchcodec использует get_frame_at() или get_frames_at()
                if hasattr(video_decoder, 'get_frame_at'):
                    result = video_decoder.get_frame_at(frame_idx)
                elif hasattr(video_decoder, 'get_frames_at'):
                    result = video_decoder.get_frames_at([frame_idx])
                    # get_frames_at возвращает список/батч
                    if isinstance(result, (list, tuple)):
                        result = result[0]
                else:
                    raise AttributeError("VideoDecoder не имеет методов get_frame_at или get_frames_at")

                # Извлекаем тензор из Frame объекта
                # В torchcodec Frame имеет атрибут data с тензором
                # HACK: На разных данных по-разному работает :/
                if hasattr(result, 'data'):
                    # Frame объект с data атрибутом
                    frame_tensor = result.data
                elif hasattr(result, 'to_tensor'):
                    # Альтернативный метод
                    frame_tensor = result.to_tensor()
                elif isinstance(result, torch.Tensor):
                    # Уже тензор
                    frame_tensor = result
                elif isinstance(result, tuple):
                    # Кортеж (frame, metadata)
                    frame_obj = result[0]
                    if hasattr(frame_obj, 'data'):
                        frame_tensor = frame_obj.data
                    else:
                        frame_tensor = frame_obj
                else:
                    # Пробуем напрямую
                    frame_tensor = result

                # Конвертируем tensor в numpy array
                if isinstance(frame_tensor, torch.Tensor):
                    frame_np = frame_tensor.cpu().numpy()

                    # Проверяем формат: обычно (C, H, W)
                    if len(frame_np.shape) == 3 and frame_np.shape[0] in [1, 3, 4]:
                        # Переставляем (C, H, W) -> (H, W, C)
                        frame_np = np.transpose(frame_np, (1, 2, 0))

                    # Нормализуем к диапазону [0, 255]
                    if frame_np.dtype == np.float32 or frame_np.dtype == np.float64:
                        if frame_np.max() <= 1.0:
                            frame_np = (frame_np * 255).astype(np.uint8)
                        else:
                            frame_np = frame_np.astype(np.uint8)
                    elif frame_np.dtype == np.uint8:
                        # Уже в правильном формате
                        pass
                    else:
                        # Другие типы - конвертируем
                        frame_np = frame_np.astype(np.uint8)

                    # Если 4 канала (RGBA), берем только RGB
                    if len(frame_np.shape) == 3 and frame_np.shape[-1] == 4:
                        frame_np = frame_np[:, :, :3]
                else:
                    frame_np = np.array(frame_tensor).astype(np.uint8)

                # Изменяем размер, сохраняя пропорции
                h, w = frame_np.shape[:2]
                new_h = int(h * (resize_width / w))
                # Делаем размеры кратными 8 (требование для Stable Diffusion)
                new_h = (new_h // 8) * 8
                new_w = (resize_width // 8) * 8

                frame_resized = cv2.resize(frame_np, (new_w, new_h))

                # Конвертируем в PIL Image
                frames.append(Image.fromarray(frame_resized))

            except Exception as e:
                print(f"\nОшибка при извлечении кадра {frame_idx}: {e}")
                # Пробуем следующий кадр
                continue

        print(f"\nУспешно извлечено {len(frames)} кадров")

        if len(frames) == 0:
            raise ValueError("Не удалось извлечь ни одного кадра!")

        return frames, target_fps

    except Exception as e:
        print(f"\n❌ Ошибка при работе с VideoDecoder: {e}")
        print("\nИнформация об объекте VideoDecoder:")
        print(f"  Тип: {type(video_decoder)}")
        print(f"  Доступные методы: {[m for m in dir(video_decoder) if not m.startswith('_')][:10]}")
        if hasattr(video_decoder, 'metadata'):
            print(f"  Метаданные: {video_decoder.metadata}")
        raise


frames, _ = extract_frames_from_decoder(ds[0]['video'])
assert len(frames) > 0, "Не удалось извлечь кадры из видео"

Исходное видео: 20.00 FPS, 200 кадров
Будет извлечено 200 кадров с шагом 1


Извлечение кадров:   0%|          | 0/200 [00:00<?, ?it/s]


Успешно извлечено 200 кадров


## Обработка видео с помощью моделей Stable Diffusion

In [7]:
from diffusers import (
    StableDiffusionControlNetPipeline,
    ControlNetModel,
    UniPCMultistepScheduler,
    StableDiffusionPipeline,
    DDIMScheduler
)
from controlnet_aux import CannyDetector, OpenposeDetector, HEDdetector


# Загрузка процессора для ControlNet:
processor = CannyDetector()
controlnet_model_id = "lllyasviel/sd-controlnet-canny"

print(f"Загружаю ControlNet модель: {controlnet_model_id}")

# Загрузка ControlNet
controlnet = ControlNetModel.from_pretrained(
    controlnet_model_id,
    torch_dtype=torch.float16
)

print("Загружаю Stable Diffusion pipeline...")

# Загрузка основной модели Stable Diffusion
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
    use_safetensors=True
)

# Оптимизация для GPU
print("Настройка оптимизаций...")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Перемещаем на GPU
pipe = pipe.to("cuda")

# Включаем оптимизации памяти
pipe.enable_xformers_memory_efficient_attention()

print("✅ Модели загружены и готовы к работе!")


/usr/local/lib/python3.12/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

Загружаю ControlNet модель: lllyasviel/sd-controlnet-canny


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

Загружаю Stable Diffusion pipeline...


...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Настройка оптимизаций...
✅ xformers включен
✅ Модели загружены и готовы к работе!


In [14]:
from typing import TypedDict


class Prompt(TypedDict):
    content: str  # Содержание промпта
    suffix: str  # Суффикс, который будет ставиться в сохраненном файле


positive_prompts = [
    Prompt(
        content="symmetrical composition, pastel color palette, vintage aesthetic, perfectly centered framing, whimsical atmosphere, retro production design, meticulous art direction, nostalgic feel, quirky characters, detailed background, soft lighting, cinematic, 35mm film",
        suffix="atmosphere"
    ),
    Prompt(
        content="black and white, high contrast, dramatic shadows, chiaroscuro lighting, 1940s aesthetic, film noir style, mysterious atmosphere, venetian blinds shadows, hard light, cinematic composition, detective movie, moody, dramatic, vintage crime thriller",
        suffix="noir"
    ),
    Prompt(
        content="1950s futuristic vision, atomic age aesthetic, streamlined design, chrome and pastel colors, ray guns and rockets, mid-century modern, space age design, optimistic future, vintage sci-fi illustration, retro technology, sleek lines, art deco influence",
        suffix="retrofuturism"
    )
]

negative_prompts = [
    Prompt(
        content="out of focus, motion blur, defocused, soft focus, bokeh mistake, unfocused subject, camera shake, blurry details, lack of sharpness, gaussian blur",
        suffix="blured"
    ),
    Prompt(
        content="warped, deformed, distorted proportions, stretched, compressed, unnatural perspective, morphed, twisted geometry, wrong anatomy, malformed structures, broken composition",
        suffix="distorted"
    )
]


# Параметры генерации
num_inference_steps = 8
guidance_scale = 7.5
controlnet_conditioning_scale = 1.0
seed = 42

In [15]:
def process_frames_with_controlnet(frames, processor, pipe, prompt, negative_prompt,
                                   num_inference_steps=20, guidance_scale=7.5,
                                   controlnet_conditioning_scale=1.0, seed=42,
                                   use_fixed_seed=True):
    """
    Обрабатывает кадры с помощью ControlNet.

    Args:
        use_fixed_seed: если True, использует один seed для всех кадров (больше согласованности)
    """
    processed_frames = []

    for i, frame in enumerate(tqdm(frames, desc="Обработка кадров")):
        # Применяем процессор ControlNet
        control_image = processor(frame)

        # Стратегия 1: Фиксированный seed для всех кадров
        # HACK: Это нужно чтобы сделать переходы между фреймами более плавными по стилю
        if use_fixed_seed:
            generator = torch.Generator(device="cuda").manual_seed(seed)
        else:
            # Стратегия 2: Близкие seed для соседних кадров
            frame_seed = seed + i
            generator = torch.Generator(device="cuda").manual_seed(frame_seed)

        # Генерируем кадр
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=control_image,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            controlnet_conditioning_scale=controlnet_conditioning_scale,
            generator=generator
        )

        processed_frames.append(output.images[0])

        # Периодически показываем прогресс
        if i % 10 == 0 and i > 0:
            print(f"Обработано {i}/{len(frames)} кадров")
            gc.collect()
            torch.cuda.empty_cache()

    return processed_frames


def generate_video(video, positive_prompt, negative_prompt):
    """Генерирует видео"""
    frames, fps = extract_frames_from_decoder(video, max_frames=120)

    processed_frames = process_frames_with_controlnet(
        frames, processor, pipe, positive_prompt, negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        seed=seed
    )

    return processed_frames

In [16]:
from google.colab import files
from uuid import uuid4


def create_video_from_frames(frames, output_path, fps=15):
    """
    Создает видео из кадров.
    """
    # Конвертируем PIL изображения в numpy массивы
    frames_np = [np.array(frame) for frame in frames]

    # Создаем видео с помощью mediapy
    media.write_video(output_path, frames_np, fps=fps)

    print(f"✅ Видео сохранено: {output_path}")
    print(f"   Длительность: {len(frames)/fps:.2f} секунд")
    print(f"   FPS: {fps}")
    print(f"   Кадров: {len(frames)}")


def gen_pipe(video, positive_prompt: Prompt, negative_prompt: Prompt, fps: int = 15) -> str:
    positive = positive_prompt.get('content')
    negative = negative_prompt.get('content')


    processed_frames = generate_video(video, positive, negative)

    pos_suffix = positive_prompt.get('suffix')
    neg_suffix = negative_prompt.get('suffix')
    output_video_path = f"{pos_suffix}_{neg_suffix}_{uuid4()}.mp4"
    create_video_from_frames(processed_frames, output_video_path, fps=fps)

    return output_video_path

In [19]:
output_video_paths = []

for video in tqdm(ds['video'][:3], desc='Обработка видео'):
    for pos, neg in tqdm(zip(positive_prompts, negative_prompts)):
        output_path = gen_pipe(video, pos, neg)
        output_video_paths.append(output_video_paths)

Обработка видео:   0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Исходное видео: 20.00 FPS, 200 кадров
Будет извлечено 120 кадров с шагом 1


Извлечение кадров:   0%|          | 0/120 [00:00<?, ?it/s]


Успешно извлечено 120 кадров


Обработка кадров:   0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

Очень долго обрабатывается :/

In [18]:
import zipfile
import glob
import os

def create_mp4_archive(output_filename='videos.zip'):
    """
    Собирает все MP4 файлы из текущей директории в ZIP-архив.

    Args:
        output_filename: имя выходного ZIP-файла

    Returns:
        Количество добавленных файлов
    """
    mp4_files = glob.glob('*.mp4')

    if not mp4_files:
        print("MP4 файлы не найдены")
        return 0

    with zipfile.ZipFile(output_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for mp4_file in mp4_files:
            print(f"Добавляем: {mp4_file}")
            zipf.write(mp4_file, os.path.basename(mp4_file))

    print(f"Создан архив {output_filename} с {len(mp4_files)} файлами")
    files.download(output_filename)
    return len(mp4_files)

create_mp4_archive('controlnet_videos.zip')

Добавляем: noir_distorted_f2f44151-8862-409d-824c-eb91aa63b103.mp4
Добавляем: atmosphere_blured_87f78884-75ae-44dd-8742-11bfa51a1189.mp4
Добавляем: atmosphere_blured_f8841855-caf1-44d1-8565-bddb97e989a2.mp4
Добавляем: atmosphere_blured_23d97556-039a-4eb8-b3c0-c4354b599fef.mp4
Добавляем: noir_distorted_8938de6e-e5cb-4ebc-934d-46c8dd1fe1a4.mp4
Добавляем: noir_distorted_c178b3bd-371d-4917-8901-9b956af13230.mp4
Создан архив controlnet_videos.zip с 6 файлами


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

6